# Bioinformatics Final Project: Group 4
### Coral Host taken into consideration is OANN.

In [17]:
import pandas as pd
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import train_test_split,cross_val_score, cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
import numpy as np
from scipy.stats import pearsonr

Loading the Gene Expression data and the meta data files for the Analysis.

In [18]:
# Loading the Gene Expression Data
Gene_expression = pd.read_csv('normalized_counts_vst.csv', index_col = 0).T
Gene_expression.head(5)

,A0A011QK89,A0A023I7E1,A0A067XMP1,A0A067YMX8,A0A075HNX4,A0A0A0RM07,A0A0B4KGY6,A0A0B6VQ48,A0A0B7P9G0,A0A0G2JDV3,...,S0EMV0,S8GJB7,V6CLA2,W0LYS5,W0TIW1,W7JLR6,W7JX98,W7K9M0,W7NDQ0,X1WER2
M_c1_C,3.907570,1.620841,3.004398,2.703474,2.986870,2.241169,2.521235,3.692186,3.004398,6.353335,...,1.620841,1.620841,1.856850,3.104000,3.119761,3.371791,1.856850,2.359892,4.727864,4.686284
M_c3_C,5.441870,1.620841,2.535626,3.657173,3.478619,2.503367,3.106804,4.619353,4.672777,7.503357,...,1.620841,1.869137,2.976699,3.426356,3.944844,4.232551,2.173351,2.273015,6.026444,6.010146
M_c5_C,5.331988,1.620841,3.125354,3.570151,3.630813,2.891829,2.891829,4.273808,5.297177,7.510740,...,1.948261,2.346861,3.570151,2.414528,3.368204,4.119319,2.729975,2.684824,5.862211,5.935485
M_c6_C,5.619051,1.974893,2.605541,3.519751,2.948207,2.404832,3.137391,4.295079,4.585000,7.673688,...,1.620841,2.544092,2.905994,2.544092,3.662709,4.478651,1.620841,2.477620,6.219767,5.997715
M_c7_C,5.021363,1.620841,2.913924,3.443582,3.627241,2.838415,2.937903,4.301555,4.861119,7.184987,...,1.887821,2.152548,3.050253,2.454656,3.666083,3.930574,2.567736,2.214481,5.709837,5.492568


In [19]:
# Loading the metadata file.
metadata = pd.read_csv("metadata.csv")
metadata.head(5)

,Sample_ID,Clade,Host,Dominant
0,M_c1_C,C,MCAV,Yes
1,M_c3_C,C,MCAV,Yes
2,M_c5_C,C,MCAV,Yes
3,M_c6_C,C,MCAV,No
4,M_c7_C,C,MCAV,Yes


Merging the data.

In [20]:
# Merging the Gene Expression and Meta data together.
data = pd.merge(Gene_expression, metadata, left_index = True, right_on = 'Sample_ID')
data.head(5)

,A0A011QK89,A0A023I7E1,A0A067XMP1,A0A067YMX8,A0A075HNX4,A0A0A0RM07,A0A0B4KGY6,A0A0B6VQ48,A0A0B7P9G0,A0A0G2JDV3,...,W0TIW1,W7JLR6,W7JX98,W7K9M0,W7NDQ0,X1WER2,Sample_ID,Clade,Host,Dominant
0,3.907570,1.620841,3.004398,2.703474,2.986870,2.241169,2.521235,3.692186,3.004398,6.353335,...,3.119761,3.371791,1.856850,2.359892,4.727864,4.686284,M_c1_C,C,MCAV,Yes
1,5.441870,1.620841,2.535626,3.657173,3.478619,2.503367,3.106804,4.619353,4.672777,7.503357,...,3.944844,4.232551,2.173351,2.273015,6.026444,6.010146,M_c3_C,C,MCAV,Yes
2,5.331988,1.620841,3.125354,3.570151,3.630813,2.891829,2.891829,4.273808,5.297177,7.510740,...,3.368204,4.119319,2.729975,2.684824,5.862211,5.935485,M_c5_C,C,MCAV,Yes
3,5.619051,1.974893,2.605541,3.519751,2.948207,2.404832,3.137391,4.295079,4.585000,7.673688,...,3.662709,4.478651,1.620841,2.477620,6.219767,5.997715,M_c6_C,C,MCAV,No
4,5.021363,1.620841,2.913924,3.443582,3.627241,2.838415,2.937903,4.301555,4.861119,7.184987,...,3.666083,3.930574,2.567736,2.214481,5.709837,5.492568,M_c7_C,C,MCAV,Yes


In [21]:
# Dropping the column that are not taken into consideration.
data = data.drop(['Sample_ID','Clade', 'Dominant' ],axis = 1)
data.head(5)

,A0A011QK89,A0A023I7E1,A0A067XMP1,A0A067YMX8,A0A075HNX4,A0A0A0RM07,A0A0B4KGY6,A0A0B6VQ48,A0A0B7P9G0,A0A0G2JDV3,...,S8GJB7,V6CLA2,W0LYS5,W0TIW1,W7JLR6,W7JX98,W7K9M0,W7NDQ0,X1WER2,Host
0,3.907570,1.620841,3.004398,2.703474,2.986870,2.241169,2.521235,3.692186,3.004398,6.353335,...,1.620841,1.856850,3.104000,3.119761,3.371791,1.856850,2.359892,4.727864,4.686284,MCAV
1,5.441870,1.620841,2.535626,3.657173,3.478619,2.503367,3.106804,4.619353,4.672777,7.503357,...,1.869137,2.976699,3.426356,3.944844,4.232551,2.173351,2.273015,6.026444,6.010146,MCAV
2,5.331988,1.620841,3.125354,3.570151,3.630813,2.891829,2.891829,4.273808,5.297177,7.510740,...,2.346861,3.570151,2.414528,3.368204,4.119319,2.729975,2.684824,5.862211,5.935485,MCAV
3,5.619051,1.974893,2.605541,3.519751,2.948207,2.404832,3.137391,4.295079,4.585000,7.673688,...,2.544092,2.905994,2.544092,3.662709,4.478651,1.620841,2.477620,6.219767,5.997715,MCAV
4,5.021363,1.620841,2.913924,3.443582,3.627241,2.838415,2.937903,4.301555,4.861119,7.184987,...,2.152548,3.050253,2.454656,3.666083,3.930574,2.567736,2.214481,5.709837,5.492568,MCAV


In [22]:
# Making the Coral HOST into numericals by making HOST == OANN as 1 and HOST != OANN as 0.
data['Host'] = (data['Host'] == 'OANN').astype(int)
data.head(5)

,A0A011QK89,A0A023I7E1,A0A067XMP1,A0A067YMX8,A0A075HNX4,A0A0A0RM07,A0A0B4KGY6,A0A0B6VQ48,A0A0B7P9G0,A0A0G2JDV3,...,S8GJB7,V6CLA2,W0LYS5,W0TIW1,W7JLR6,W7JX98,W7K9M0,W7NDQ0,X1WER2,Host
0,3.907570,1.620841,3.004398,2.703474,2.986870,2.241169,2.521235,3.692186,3.004398,6.353335,...,1.620841,1.856850,3.104000,3.119761,3.371791,1.856850,2.359892,4.727864,4.686284,0
1,5.441870,1.620841,2.535626,3.657173,3.478619,2.503367,3.106804,4.619353,4.672777,7.503357,...,1.869137,2.976699,3.426356,3.944844,4.232551,2.173351,2.273015,6.026444,6.010146,0
2,5.331988,1.620841,3.125354,3.570151,3.630813,2.891829,2.891829,4.273808,5.297177,7.510740,...,2.346861,3.570151,2.414528,3.368204,4.119319,2.729975,2.684824,5.862211,5.935485,0
3,5.619051,1.974893,2.605541,3.519751,2.948207,2.404832,3.137391,4.295079,4.585000,7.673688,...,2.544092,2.905994,2.544092,3.662709,4.478651,1.620841,2.477620,6.219767,5.997715,0
4,5.021363,1.620841,2.913924,3.443582,3.627241,2.838415,2.937903,4.301555,4.861119,7.184987,...,2.152548,3.050253,2.454656,3.666083,3.930574,2.567736,2.214481,5.709837,5.492568,0


The HOST is our Target and the rest of the data is the inputs.

In [23]:
# Target column
y = data['Host']
# Inputs of the target column
X = data.iloc[:, :-1]

In [25]:
# Splitting the data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 41)

## Logistic Regression

In [27]:
# Cross Validation 
# Defining the cross validation accuracy.
lr = LogisticRegression(max_iter = 1000)

# Cross Validation Accuracy with a fold of 5.
cv_score = cross_val_score(lr, X_train, y_train, cv = 5, scoring = 'accuracy')
print("Logistic Regression CV Accuracy:", cv_score.mean())

Logistic Regression CV Accuracy: 0.9380952380952381


Fitting the model with all parameters

In [29]:
# FItting the model.
lr.fit(X_train, y_train)

# Predicting the results on the testing set.4
y_pred = lr.predict(X_test)

# FInding the Accurayc of the logistic Reqgression Model.
Accuracy = accuracy_score(y_test, y_pred)

print("Logistic Regression Testing Accuracy:", Accuracy)

Logistic Regression Testing Accuracy: 1.0


Creating the confusion Matrix of the predictions

In [30]:
from sklearn.metrics import confusion_matrix
from tabulate import tabulate

# Assuming y_test and y_pred are already defined
cons_matrix = confusion_matrix(y_test, y_pred)

# Calculate the error rate
error_rate = (cons_matrix[0, 1] + cons_matrix[1, 0]) / cons_matrix.sum()

# Prepare table data
table_data = [
    ["Actual Positive", cons_matrix[0, 0], cons_matrix[0, 1]],
    ["Actual Negative", cons_matrix[1, 0], cons_matrix[1, 1]]
]

# Define headers
headers = ["Confusion Matrix:", "Predicted Positive", "Predicted Negative"]

print(tabulate(table_data, headers, tablefmt = "grid"))
print(f"Error Rate: {error_rate:.4f}")

Confusion Matrix:
+-----------------+----------------------+----------------------+
|                 |   Predicted Positive |   Predicted Negative |
+=================+======================+======================+
| Actual Positive |                    7 |                    0 |
+-----------------+----------------------+----------------------+
| Actual Negative |                    0 |                    2 |
+-----------------+----------------------+----------------------+
Error Rate: 0.0000


In [26]:
# Fit the logistic regression model
lr.fit(X_train, y_train)

# Get the coefficients and corresponding feature names
coefficients = lr.coef_[0]
feature_names = X_train.columns

# Create a DataFrame to display coefficients and feature names
coefficients_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})

# Display the DataFrame sorted by absolute coefficient values
coefficients_df['Absolute Coefficient'] = coefficients_df['Coefficient'].abs()
sorted_coefficients = coefficients_df.sort_values(by='Absolute Coefficient', ascending=False)

# Display the top features
top_features = sorted_coefficients.head(10)  # Adjust the number as needed
print("Top Features:")
print(top_features)

Logistic Regression Cross-validated Accuracy: 0.9380952380952381
Logistic Regression Test Set Accuracy: 1.0
Confusion Matrix:
[[7 0]
 [0 2]]
Error Rate: 0.0000
Top Features:
     Feature  Coefficient  Absolute Coefficient
2349  Q7PHR1    -0.028657              0.028657
617   O88764     0.025619              0.025619
318   F4HQD4     0.025353              0.025353
2752  Q8ZRS8     0.025322              0.025322
2032  Q5RE15     0.023800              0.023800
3270  Q9SF16     0.023698              0.023698
2473  Q86WI1    -0.023338              0.023338
2815  Q94B38     0.023211              0.023211
2417  Q80TF6     0.022660              0.022660
2214  Q6H6D2     0.022637              0.022637


# Logistic Regression 

## Questions 

### How Many Genes/Components Are Obtained/Required Significant Results? 
  In this question, based on your algorithm, are there a certain number of genes or components (depending on the algorithm) that can perform the classification of your problem? 

### What is the Error Rate of These Algorithms? 
  In this question, based on your algorithms, how much of the test dataset is misclassified?  
  
### What are two genes that were signficiantly correlated to your problem? 
  In this question, based on your algorithm, identify and annotate 2 genes using [uniprot](uniprot.org). Do you think they are biologically relevant? 

### What can we infer about this dataset based on these results? 
  In this question, you should think about the overall classification results of this algorithm. Does the type of algorithm used here tell you anything about the data in this study? What is biologically meaningful about this? 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

correlation_matrix = X_train.corr()
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Correlation Matrix")
plt.show()